In [1]:
import pandas as pd
import numpy as np

import os

In [5]:
TRANSFORM_FOLDER = '../../DATA/TRANSFORM/entity_details/'
FINAL_FOLDER = '../../DATA/FINAL'

In [6]:
# files = os.listdir(TRANSFORM_FOLDER)
files = [
 'financial_statements_register_entity_details.csv',
 'insolvency_register.csv',
 'vat_registration_cancellation.csv']

In [7]:
data = dict()
data['entity_details'] = pd.read_csv(TRANSFORM_FOLDER + 'financial_statements_register_entity_details.csv')
data['insolvency_register'] = pd.read_csv(TRANSFORM_FOLDER + 'insolvency_register.csv')
data['vat_registration_cancellation'] = pd.read_csv(TRANSFORM_FOLDER + 'vat_registration_cancellation.csv')

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (13,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# most_recent_business_register_record = data['business_register'].copy().sort_values(['valid_to', 'valid_from']).groupby(['entity_ico']).tail(1)
# most_recent_business_register_record.drop(columns=['stat_codes_actualization', 'main_activity_codelist_code', 'esa2010_codelist_code', 'export_date', 'id'], inplace=True)
# most_recent_business_register_record['valid_to_year'] = most_recent_business_register_record['valid_to'].apply(lambda x: np.nan if x == '2023-12-12' else pd.to_datetime(x).year)
# most_recent_business_register_record = most_recent_business_register_record[['entity_ico', 'valid_to_year']]
# most_recent_business_register_record.query('valid_to_year == valid_to_year and valid_to_year > 1800', inplace=True)
# most_recent_business_register_record['event_type'] = 'RO_valid_to'
# most_recent_business_register_record.rename(columns={'valid_to_year':'year'},inplace=True)
# most_recent_business_register_record

In [9]:
insolvency_info = data['insolvency_register'].copy()
insolvency_info['proceeding_type'] = insolvency_info['proceeding_type'].apply(lambda x: 'ostatne_konania' if x not in ['konkurz', 'restrukturalizacia '] else x)
insolvency_info

insolvency_info.drop_duplicates(['entity_ico', 'proceeding_type', 'proceeding_month'], inplace=True)

insolvency_info = insolvency_info.sort_values('proceeding_month', ascending=False).groupby(['entity_ico', 'proceeding_type', 'proceeding_month']).tail(1)

insolvency_info.rename(columns={'proceeding_type': 'event_type', 'proceeding_month': 'year'}, inplace=True)

insolvency_info = insolvency_info[['entity_ico', 'year', 'event_type']]

insolvency_info['event_type'] = insolvency_info['event_type'].apply(lambda x: 'RU_' + x)
insolvency_info


,entity_ico,year,event_type
0,53051505.0,2023,RU_konkurz
276,45380058.0,2023,RU_konkurz
287,45257337.0,2023,RU_konkurz
286,35964065.0,2023,RU_konkurz
285,44353260.0,2023,RU_konkurz
...,...,...,...
7033,31105424.0,1993,RU_ostatne_konania
7032,31655785.0,1993,RU_ostatne_konania
6995,598321.0,1993,RU_ostatne_konania
7019,31412874.0,1993,RU_ostatne_konania


In [10]:
vat_registration_cancellation_info = data['vat_registration_cancellation'].copy()
# vat_registration_cancellation_info

vat_registration_cancellation_info['entity_ico'] = pd.to_numeric(vat_registration_cancellation_info['entity_ico'], errors = 'coerce')
vat_registration_cancellation_info.query('entity_ico == entity_ico', inplace=True)
vat_registration_cancellation_info = vat_registration_cancellation_info[['entity_ico', 'vat_cancellation_year']].drop_duplicates(['entity_ico'])
vat_registration_cancellation_info['vat_cancellation_year'] = pd.to_numeric(vat_registration_cancellation_info['vat_cancellation_year'])
vat_registration_cancellation_info['event_type'] = 'DPHZ_vat_registration_cancelled'
vat_registration_cancellation_info.rename(columns = {'vat_cancellation_year': 'year'}, inplace=True)
vat_registration_cancellation_info

,entity_ico,year,event_type
0,37712080.0,2020,DPHZ_vat_registration_cancelled
1,37257455.0,2018,DPHZ_vat_registration_cancelled
2,37713256.0,2013,DPHZ_vat_registration_cancelled
3,35125730.0,2020,DPHZ_vat_registration_cancelled
4,37338048.0,2020,DPHZ_vat_registration_cancelled
...,...,...,...
4749,2083477.0,2018,DPHZ_vat_registration_cancelled
4750,50635808.0,2023,DPHZ_vat_registration_cancelled
4751,6199917.0,2020,DPHZ_vat_registration_cancelled
4752,53714521.0,2023,DPHZ_vat_registration_cancelled


In [11]:
# vat_registration_cancellation_info.merge(data['entity_details'], on = 'entity_ico', how = 'left').query('id == id')

In [12]:
ru_entity_details = data['entity_details'].copy()
ru_entity_details.query('financial_statements_list == financial_statements_list', inplace= True)

ru_entity_details = (ru_entity_details
                     .merge(insolvency_info, on = 'entity_ico', how = 'left')
                     .merge(vat_registration_cancellation_info, on = 'entity_ico', how = 'left'))


ru_entity_details['establishment_date'] = pd.to_datetime(ru_entity_details['establishment_date'], errors='coerce')
ru_entity_details['cancellation_date'] = pd.to_datetime(ru_entity_details['cancellation_date'], errors='coerce')
ru_entity_details['cancellation_date_RO'] = pd.to_datetime(ru_entity_details['cancellation_date_RO'], errors='coerce')

ru_entity_details = ru_entity_details.query('establishment_date == establishment_date and id not in [1019370, 1063773, 1068320]')

ru_entity_details = ru_entity_details[['entity_ico', 'establishment_date', 'cancellation_date', 'cancellation_date_RO']]

ru_entity_details['establishment_year'] = ru_entity_details['establishment_date'].apply(lambda x: x.year if x==x else np.nan)
ru_entity_details['cancellation_year'] = ru_entity_details['cancellation_date'].apply(lambda x: x.year if x==x else np.nan)
ru_entity_details['cancellation_year_RO'] = ru_entity_details['cancellation_date_RO'].apply(lambda x: x.year if x==x else np.nan)

ru_entity_details = ru_entity_details[['entity_ico', 'establishment_year', 'cancellation_year', 'cancellation_year_RO']]
ru_entity_details.columns = ['entity_ico', 'RUZ_established', 'RUZ_cancelled', 'RO_cancelled']
ru_entity_events = ru_entity_details.melt(id_vars='entity_ico')
ru_entity_events.columns = ['entity_ico', 'event_type', 'year']
ru_entity_events.query('year == year', inplace = True)
ru_entity_events['year'] = ru_entity_events['year'].apply(lambda x: x if isinstance(x, float) else x.year)
ru_entity_events

,entity_ico,event_type,year
0,603783.0,RUZ_established,1991.0
1,590797.0,RUZ_established,1990.0
2,2313.0,RUZ_established,1992.0
3,677957.0,RUZ_established,1992.0
4,615439.0,RUZ_established,1991.0
...,...,...,...
706969,52425576.0,RO_cancelled,2021.0
707009,52618382.0,RO_cancelled,2021.0
707059,52835383.0,RO_cancelled,2022.0
707201,53367600.0,RO_cancelled,2021.0


In [13]:
entity_issues_events_long = (pd.concat(
    [
         insolvency_info,
         ru_entity_events,
         vat_registration_cancellation_info
    ], axis = 0))

entity_issues_events_long

,entity_ico,year,event_type
0,53051505.0,2023.0,RU_konkurz
276,45380058.0,2023.0,RU_konkurz
287,45257337.0,2023.0,RU_konkurz
286,35964065.0,2023.0,RU_konkurz
285,44353260.0,2023.0,RU_konkurz
...,...,...,...
4749,2083477.0,2018.0,DPHZ_vat_registration_cancelled
4750,50635808.0,2023.0,DPHZ_vat_registration_cancelled
4751,6199917.0,2020.0,DPHZ_vat_registration_cancelled
4752,53714521.0,2023.0,DPHZ_vat_registration_cancelled


In [14]:
valid_entities_list = list(entity_issues_events_long.query('event_type == "RUZ_established"')['entity_ico'])
# valid_entities_list

In [15]:
entity_issues_events_long.query('event_type not in ["RO_valid_to", "RUZ_established"] and entity_ico in @valid_entities_list')['event_type'].value_counts()

RUZ_cancelled                      55828
RU_konkurz                          4803
DPHZ_vat_registration_cancelled     3816
RO_cancelled                        2200
RU_ostatne_konania                   732
Name: event_type, dtype: int64

In [17]:
entity_issues_events_long['indicator'] = 1
entity_issues_events_long.to_csv('../../DATA/TRANSFORM/issues_events.csv', index = False)

In [18]:
entity_issues_events = pd.pivot_table(entity_issues_events_long, index = ['entity_ico', 'year'], columns = 'event_type', values='indicator', fill_value=0)
# entity_issues_events.query('entity_ico in @valid_entities_list', inplace = True)
entity_issues_events.reset_index(inplace=True)
entity_issues_events

event_type,entity_ico,year,DPHZ_vat_registration_cancelled,RO_cancelled,RUZ_cancelled,RUZ_established,RU_konkurz,RU_ostatne_konania
0,0.000000e+00,2008.0,0,0,0,0,1,0
1,0.000000e+00,2017.0,0,0,0,0,0,1
2,3.500000e+01,2004.0,0,0,0,0,0,1
3,6.710000e+02,1970.0,0,0,0,1,0,0
4,6.710000e+02,2018.0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
415770,6.072081e+07,2011.0,0,0,0,0,1,0
415771,6.398754e+07,1999.0,0,0,0,1,0,0
415772,3.647911e+08,2013.0,0,0,0,0,0,1
415773,4.488678e+08,2018.0,0,0,0,0,1,0


In [19]:
entity_issues_events_1y_offset = entity_issues_events.copy()
entity_issues_events_2y_offset = entity_issues_events.copy()
entity_issues_events_1y_offset['year'] -= 1
entity_issues_events_2y_offset['year'] -= 2

entity_issues_events_1y_offset.columns = ['entity_ico', 'year'] + [col + '_1y_off' for col in list(entity_issues_events_1y_offset.columns)[2:]]

entity_issues_events_2y_offset.columns = ['entity_ico', 'year'] + [col + '_2y_off' for col in list(entity_issues_events_2y_offset.columns)[2:]]

entity_issues_indicators = (entity_issues_events
                        .merge(entity_issues_events_1y_offset, on = ['entity_ico', 'year'], how = 'outer')
                        .merge(entity_issues_events_2y_offset, on = ['entity_ico', 'year'], how = 'outer'))


entity_issues_indicators['entity_ico'] = entity_issues_indicators['entity_ico'].apply(int)

for col in entity_issues_indicators.columns:
    entity_issues_indicators[col].fillna(0, inplace=True)

entity_issues_indicators

,entity_ico,year,DPHZ_vat_registration_cancelled,RO_cancelled,RUZ_cancelled,RUZ_established,RU_konkurz,RU_ostatne_konania,DPHZ_vat_registration_cancelled_1y_off,RO_cancelled_1y_off,RUZ_cancelled_1y_off,RUZ_established_1y_off,RU_konkurz_1y_off,RU_ostatne_konania_1y_off,DPHZ_vat_registration_cancelled_2y_off,RO_cancelled_2y_off,RUZ_cancelled_2y_off,RUZ_established_2y_off,RU_konkurz_2y_off,RU_ostatne_konania_2y_off
0,0,2008.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2017.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,35,2004.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,671,1970.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,671,2018.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240547,60720808,2009.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1240548,63987538,1997.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1240549,364791128,2011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1240550,448867799,2016.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
entity_issues_indicators = entity_issues_indicators.query('entity_ico > 0')[['entity_ico', 'year'] + sorted(list(entity_issues_indicators.columns)[2:])].sort_values(['entity_ico', 'year'])

In [21]:
entity_issues_indicators.to_csv('../../DATA/TRANSFORM/issues_indicators.csv', index = False)